In [29]:
from dask.distributed import Client, Executor, progress
# e = Executor('tcp://10.15.197.146:8786')
e = Executor('localhost:8786')
# e = Executor('0.0.0.0:9786')

e

<Client: scheduler='tcp://localhost:8786' processes=2 cores=8>

In [39]:
from itertools import product
import dask.dataframe as dd
from dask import delayed
import pandas as pd
import pandas as pd
import numpy as np
import random
import numpy as np

def fake_data(size):
    return pd.DataFrame({'product': [''.join(random.sample(alphabet, 4)) for i in range(size)],
                         'value': np.random.exponential(100, size=size).astype(int),
                         'rating': np.random.normal(0, 1, size=size)})

In [37]:

alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
products = [''.join(x) for x in product(alphabet, alphabet, alphabet, alphabet)]

dimension = pd.DataFrame({'product': products, 
                          'category': np.random.randint(0, 1000, size=len(products))})
example = fake_data(5)
example.merge(dimension, on='product', how='inner')

partitions = []
for i in range(10):
    partitions.append(delayed(fake_data)(10000))
    
example = fake_data(1)

df = dd.from_delayed(partitions, example)

In [40]:
joined = df.merge(dimension, how='inner', on='product')
joined.head()

,product,rating,value,category
0,EQMP,0.866543,124,115
1,CPYR,1.070692,16,86
2,BECA,-0.307197,99,135
3,XQIB,-0.092523,125,674
4,IPOE,-2.017492,7,127


In [41]:
joined.groupby('category').rating.mean().nlargest(10).compute()

category
435    0.414251
886    0.328292
487    0.306072
338    0.267605
381    0.258749
394    0.256630
546    0.254954
349    0.230423
147    0.228695
588    0.228511
Name: rating, dtype: float64

In [42]:
dfs = [delayed(fake_data)(10000) for i in range(1000)]
example = fake_data(1)
df = dd.from_delayed(dfs, example)

In [43]:
df

,product,rating,value
npartitions=1000,,,
None,object,float64,int64
None,...,...,...
...,...,...,...
None,...,...,...
None,...,...,...


In [44]:
joined = df.merge(dimension, how='inner', on='product')
result = joined.groupby('category').rating.mean().nlargest(10)
future = e.compute(result)
progress(future)